# TMDB x IMDb 영화 리뷰 데이터 수집

1. 수집 대상: TMDB x IMDb 영화 데이터
2. 수집 기간: 2020년 1월 1일 ~ 2025년 11월 30일(6년치)
3. 수집 범위: TMDB 영화 데이터의 IMDb 리뷰 전체 페이지 (TMDB 데이터의 imdb_id 기준으로 수집)
4. 수집 방식: 공식 API 기반 크롤링 (GraphQL 엔드포인트를 통해 AJAX 리뷰 페이지네이션을 트리거해서 전체 리뷰 수집)
  - GraphQL: 서버에게 원하는 데이터만 선택해서 달라고 요청할 수 있는 API 방식 (예: 리뷰에서 username, text, rating만 가져와 줌)
  - AJAX: 페이지 새로고침 없이 서버에서 JSON을 받아오는 기술 (IMDb 리뷰 로딩이 AJAX 기반)
5. 수집 컬럼(10개): 
imdb_id
author_rating
review_title
review_text
review_id
username
user_id
review_date
helpful_up_votes
helpful_down_votes
6. 수집 결과: 총 1,099,893개 수집 완료, 약 396분(6.6시간) 소요

#### 1단계: 최종 영화 데이터에서 release_year 만들기

In [1]:
import pandas as pd

# 1) 최종 영화 데이터 로드
df = pd.read_csv("/Users/cha/Downloads/최종 프로젝트/데이터셋 후보/API & 크롤링/승윤 리뷰 수집 코드/data/movies_with_metascore.csv")

# 2) release_year 컬럼 만들기 (문자열 → datetime → 연도)
df["release_year"] = pd.to_datetime(df["release_date"], errors="coerce").dt.year

# 3) 연도 분포 한번 확인 (2005~2025가 잘 들어있는지)
df["release_year"].value_counts().sort_index()


release_year
2005     821
2006     981
2007    1000
2008    1067
2009    1166
2010    1153
2011    1275
2012    1309
2013    1493
2014    1600
2015    1514
2016    1734
2017    1855
2018    1854
2019    1883
2020    1445
2021    1438
2022    1529
2023    1350
2024    1077
2025     572
Name: count, dtype: int64

#### 2단계: 내 담당 구간(2020~2025)만 필터링해서 CSV로 저장


In [2]:
# 4) 2020~2025년 필터
df_0509 = df[
    (df["release_year"] >= 2020) &
    (df["release_year"] <= 2025)
].copy()

print("내 담당 영화 개수:", len(df_0509))
print(df_0509[["title", "release_date", "release_year", "imdb_id"]].head())

# 5) 내 담당용 CSV 저장
df_0509.to_csv(
    "../data/movies_2020_2025_for_reviews.csv",
    index=False,
    encoding="utf-8-sig"
)


내 담당 영화 개수: 7411
                     title release_date  release_year     imdb_id
20705        The Gentlemen   2020-01-01          2020   tt8367814
20706        Ghost Stories   2020-01-01          2020  tt10230404
20707            Tolo Tolo   2020-01-01          2020  tt10413566
20708  Turu, the Wacky Hen   2020-01-01          2020   tt8790552
20709                 Coma   2020-01-01          2020   tt6087226


In [1]:
import sys
sys.path.append("..")  # 노트북이 notebooks/에 있을 때만 필요

from data_collector.imdb_reviews import crawl_imdb_reviews

await crawl_imdb_reviews(
    input_csv_path="../data/movies_2020_2025_for_reviews.csv",   # 아까 만든 7411편 리스트
    output_csv_path="../data/imdb_reviews_2020_2025_full.csv",   # 최종 리뷰 파일
    max_titles=None,               # 전체 7411편 다 돌리기
    max_reviews_per_title=None     # 영화당 리뷰 전부 수집
)


🚀 IMDb 리뷰 크롤링 시작
📂 입력: ../data/movies_2020_2025_for_reviews.csv
💾 출력: ../data/imdb_reviews_2020_2025_full.csv
📌 기존 리뷰 파일 발견 → 이미 수집된 imdb_id 스킵
   이미 수집된 영화 수: 1546
🎯 이번에 수집할 영화 수: 5865
➖ Las sirvientas calientes 4 (tt12935210) 리뷰 없음
➖ Death Can Wait (tt12263440) 리뷰 없음
➖ Forky Asks a Question: What Is Cheese? (tt11194650) 리뷰 없음
➖ The League of Five (tt10149080) 리뷰 없음
➖ Queens of The Field (tt9031048) 리뷰 없음
➖ Everyone Is There (tt14184472) 리뷰 없음
➖ Patricia, A Hidden Passion (tt10963872) 리뷰 없음
➖ I Wish Someone Were Waiting for Me Somewhere (tt9151382) 리뷰 없음
➖ Our Way To The Sea (tt10702006) 리뷰 없음
➖ Sí, Mi Amor (tt11734696) 리뷰 없음
📊 진행 상황: 10/5865 (경과 0.1분)
➖ Ducoboo 3 (tt10658928) 리뷰 없음
➖ Luccas Neto em: Acampamento de Férias 2 (tt12325326) 리뷰 없음
➖ Red Latex (tt7852780) 리뷰 없음
➖ Permette? Alberto Sordi (tt11800888) 리뷰 없음
➖ Papi Sitter (tt10746876) 리뷰 없음
➖ Lucky (tt10966156) 리뷰 없음
➖ Let It Be Law (tt10208192) 리뷰 없음
➖ Dragons: Rescue Riders: Hunt for the Golden Dragon (tt11875456) 리뷰 없음
➖ Fa

In [2]:
import pandas as pd

df_full = pd.read_csv("../data/imdb_reviews_2020_2025_full.csv")
print(df_full.shape)
df_full.head()

# 영화별 리뷰 개수
df_full["imdb_id"].value_counts().head()


(1099893, 11)


imdb_id
tt1877830     8351
tt12361974    8153
tt8110330     7707
tt7126948     7327
tt10872600    6201
Name: count, dtype: int64

In [3]:
df_full

,author_rating,review_title,review_text,review_id,username,user_id,review_date,helpful_up_votes,helpful_down_votes,imdb_id,review_text_length
0,8.0,Watch this for a few minutes of Colin Farrell,Classic Guy Ritchie movie. Never a dull moment...,rw9101966,prashantmannur,ur53260520,2023-06-05,71,4,tt8367814,606
1,9.0,"I don't understand the critics, this is brilli...","The writing, directing, acting and the general...",rw5376548,watch_umean,ur78005679,2020-01-03,1568,235,tt8367814,194
2,9.0,Ritchie returns to the crime genre and does so...,Has there ever been a bad performance in a Guy...,rw5371151,jtindahouse,ur3793011,2020-01-01,1051,161,tt8367814,1890
3,9.0,That's Hugh Grant?! It seems so unlike him in ...,"Aside from a few very marketable films like ""S...",rw6090726,planktonrules,ur2467618,2020-09-13,163,23,tt8367814,1825
4,8.0,Guy's Best Since Snatch.,"It's Guy Ritchies Best since Snatch, it's wher...",rw5372497,SimonMarkland,ur43437598,2020-01-01,643,130,tt8367814,453
...,...,...,...,...,...,...,...,...,...,...,...
1099888,9.0,Pleasantly Surprised!,I honestly didn't expect much when watching th...,rw10996217,crosemeepy,ur211473755,2025-12-03,0,0,tt26443597,486
1099889,10.0,One of the best sequels ever,Zootopia 2 is top-tier cinema because it doubl...,rw10996817,Iuliap-515,ur211478155,2025-12-03,0,0,tt26443597,315
1099890,5.0,The Dub Change Is a Dealbreaker (LATAM Review),Huge bummer for Latin American fans: they swap...,rw10990037,cesar_gonzomont,ur26245207,2025-11-30,0,1,tt26443597,307
1099891,10.0,On par with or better than the first.,I loved seeing the passion and talent from the...,rw10993830,Foxjump,ur26966336,2025-12-01,0,0,tt26443597,362


In [5]:
# 결측률 확인
df_full.isna().mean().sort_values(ascending=False)

author_rating         0.018796
review_title          0.000024
review_text           0.000000
review_id             0.000000
username              0.000000
user_id               0.000000
review_date           0.000000
helpful_up_votes      0.000000
helpful_down_votes    0.000000
imdb_id               0.000000
review_text_length    0.000000
dtype: float64